# Bus Coloring

From the thesis, p.39f.

Loads the IEEE14 model, identifies each connected set of bus components and colors connected buses in the same color.

In [ ]:
import networkx as nx
import random

from pyapi_rts.api import *
from pyapi_rts.generated.BUS import BUS
from pyapi_rts.generated.rtdssharcsldBUSLABEL import rtdssharcsldBUSLABEL
from pyapi_rts.generated.enums.ScolEnumParameter import ScolEnum

fx = Draft()
fx.read_file("ieee14.dfx")

G = fx._subsystems[0].get_connection_graph()
SG = G.subgraph([n for n, attrdict in G.nodes(data=True) if not "3P2W" in attrdict['type']])

start_nodes = [n for n, ad in G.nodes(data=True) if ad['type'] == 'rtds_sharc_sld_BUSLABEL']
bus_list = fx.get_components_by_type("BUS", False)

colors = list(ScolEnum)
colors.remove(ScolEnum.WHITEWHITE)

for start in start_nodes:
    id_list = list(nx.dfs_postorder_nodes(SG, source=start))
    col = random.choice(colors)
    count = 0
    for bus in bus_list:
        if bus.uuid in id_list:
            bus: BUS = bus
            bus.CONFIGURATION.SCOL.set_value(col)
            count += 1
            fx._subsystems[0].modify_component(bus)
    buslabel: rtdssharcsldBUSLABEL = fx._subsystems[0].get_by_id(
        start, False)
    buslabel.Parameters.COL.set_value(col)
    print(f"Found {count} buses, coloring with {col}")
    fx._subsystems[0].modify_component(buslabel)

fx.write_file("ieee_out.dfx")

## Only Color Buses Directly Connected to Buslabel

Using get_connected_at_point()

In [ ]:
import random

from pyapi_rts.api import *
from pyapi_rts.generated.BUS import BUS
from pyapi_rts.generated.enums.ScolEnumParameter import ScolEnum

colors = list(ScolEnum)
colors.remove(ScolEnum.WHITEWHITE)

draft = Draft()
draft.read_file("ieee14.dfx")

buslabels = draft.get_components_by_type("rtds_sharc_sld_BUSLABEL", True)
for label in buslabels:
    color = random.choice(colors)
    label.Parameters.COL.set_value(color)
    draft.subsystems[0].modify_component(label)
    for connected_comp in label.parent.get_connected_at_point(label.uuid, "A1", return_connecting=True):
        if isinstance(connected_comp, BUS):
            connected_comp.CONFIGURATION.SCOL.set_value(color)
            connected_comp.parent.modify_component(connected_comp)

draft.write_file("ieee_out_new.dfx")